# Modelling and Estimation
We are proceeding to perform causal inference as if the necessary assumptions are met. Neal explains that we can identify causal effects with the "adjustment formula": $$E[Y(1) - Y(0)] = E_X[E[Y|T=1, X] - E[Y|T=0, X]]  $$
We can estimate the quantities $E[Y|T=1, X]$ and $E[Y|T=0, X]$ from the data by using a machine learning model. We will first use linear regression and then attempt to use a more advanced model. The general approach for this code is based on Neal(2020) and VanderPlas(2016). For a basis to use linear regression for a binary response see Gomila(2021).

In [53]:
# imports
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression

mpl.style.use('ggplot')

# load data
diabetes = pd.read_csv('../data/diabetes_binary_health_indicators_BRFSS2015.csv')
diabetes.head()

,Diabetes_binary,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,...,AnyHealthcare,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income
0,0.0,1.0,1.0,1.0,40.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,5.0,18.0,15.0,1.0,0.0,9.0,4.0,3.0
1,0.0,0.0,0.0,0.0,25.0,1.0,0.0,0.0,1.0,0.0,...,0.0,1.0,3.0,0.0,0.0,0.0,0.0,7.0,6.0,1.0
2,0.0,1.0,1.0,1.0,28.0,0.0,0.0,0.0,0.0,1.0,...,1.0,1.0,5.0,30.0,30.0,1.0,0.0,9.0,4.0,8.0
3,0.0,1.0,0.0,1.0,27.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,2.0,0.0,0.0,0.0,0.0,11.0,3.0,6.0
4,0.0,1.0,1.0,1.0,24.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,2.0,3.0,0.0,0.0,0.0,11.0,5.0,4.0


# Linear Regression Modelling

In [54]:
# split up feature matrix and target array
# X_treat matrix includes the treatment variable smoker 
X_treat =  diabetes.drop('Diabetes_binary', axis = 1)
y = diabetes['Diabetes_binary']

In [55]:
# set up and fit regression model
model = LinearRegression(fit_intercept= True)
model.fit(X_treat,y)

print(model.intercept_)
print(model.coef_)

-0.2982880332286123
[ 7.52952909e-02  5.59937459e-02  4.35893059e-02  6.87790112e-03
 -5.89882654e-03  3.74018864e-02  6.74262614e-02 -6.84861665e-03
 -1.68027818e-03 -2.78144759e-03 -5.07535115e-02  1.50586175e-02
 -7.33412971e-03  4.78636266e-02 -6.04433848e-04  1.08423185e-05
  4.43124383e-02  1.67516237e-02  7.31037997e-03 -3.30012545e-03
 -6.34484438e-03]


In the adjustment formula we take the expected value with respect to $X$ of $[E[Y|T=1, X] - E[Y|T=0, X]]$. In order to estimate this expected value we need to average over the estimates of $[E[Y|T=1, X] - E[Y|T=0, X]]$ for every $X$ value in the data set. Hence we need to first obtain estimates of $E[Y|T=1, X]$ and $E[Y|T=0, X]$ for each $X$ value. For this we use our model predictions.

In [56]:
# X_treat1 indicates that we are setting the treatment variable to 1
X_treat1 = X_treat.copy()
X_treat1['Smoker'] = 1

# X_treat0 indicates that we are setting the treatment variable to 0
X_treat0 = X_treat.copy()
X_treat0['Smoker'] = 0

# obtain predictions
predict1 = model.predict(X_treat1)
predict0 = model.predict(X_treat0)

In [57]:
predict1

array([ 0.46455648,  0.02219586,  0.33697391, ..., -0.02819253,
        0.17215605,  0.24562523])

In [58]:
predict0

array([ 0.47045531,  0.02809469,  0.34287274, ..., -0.0222937 ,
        0.17805487,  0.25152405])

Note that these are predictions of the expected value of the Bernoulli diabetes random variable, i.e. the probability of diabetes given the covariates. These are not classifications.

Now that we have estimates for $E[Y|T=1, X]$ and $E[Y|T=0, X]$ we can take the average of their difference to give an estimate of the ATE.

In [59]:
e_ate= np.mean(predict1 - predict0)
e_ate.round(4)

-0.0059

## Observations
So our estimated ATE is very close to zero. It is actually significantly closer to zero than the estimated association, which was 0.04. Under our assumptions, this could be interpreted that even though there is a minor association positive association between smoking and diabetes, this can be attributed to other confounding factors. For example, smokers may also have poor eating habits, a known diabetes risk factor([https://www.cdc.gov/diabetes/basics/risk-factors.html](https://www.cdc.gov/diabetes/basics/risk-factors.html)). However, under said assumptions, our analysis indicates that there is almost no direct causal effect between smoking and diabetes. 

# BART
It seems more practical to implement BART in R.

# Type 2 Diabetes Analysis

In [60]:
# subset data, drop Sex column since no longer variable
diabetes2 = diabetes.loc[(diabetes.Age >= 3) & (diabetes.Sex == 1),:].drop('Sex', axis = 1) 

## Linear Regression Modelling

In [61]:
# split up feature matrix and target array
# X_treat matrix includes the treatment variable smoker 
d2_X_treat =  diabetes2.drop('Diabetes_binary', axis = 1)
d2_y = diabetes2['Diabetes_binary']

# set up and fit regression model
model = LinearRegression(fit_intercept= True)
model.fit(d2_X_treat,d2_y)

print(model.intercept_)
print(model.coef_)

-0.39212555241153985
[ 0.07174747  0.04986611  0.0484847   0.00813693 -0.00297228  0.03634893
  0.05982382 -0.00655108 -0.00872993 -0.00111087 -0.06686805  0.01904676
 -0.00513399  0.05344925 -0.0005163  -0.00011169  0.0527512   0.01296222
 -0.00114239 -0.00627626]


In [62]:
# X_treat1 indicates that we are setting the treatment variable to 1
d2_X_treat1 = d2_X_treat.copy()
d2_X_treat1['Smoker'] = 1

# X_treat0 indicates that we are setting the treatment variable to 0
d2_X_treat0 = d2_X_treat.copy()
d2_X_treat0['Smoker'] = 0

# obtain predictions
predict1 = model.predict(d2_X_treat1)
predict0 = model.predict(d2_X_treat0)

In [63]:
e_ate= np.mean(predict1 - predict0)
e_ate.round(4)

-0.003

**Not much different from our original analysis.**

# Logistic Regression Analysis

Like before, we first use all the data.

In [64]:
# split up feature matrix and target array
# X_treat matrix includes the treatment variable smoker 
X_treat =  diabetes.drop('Diabetes_binary', axis = 1)
y = diabetes['Diabetes_binary']

# fit logistic regression model
logistic = LogisticRegression(solver='liblinear')
logistic.fit(X_treat, y)

# X_treat1 indicates that we are setting the treatment variable to 1
X_treat1 = X_treat.copy()
X_treat1['Smoker'] = 1

# X_treat0 indicates that we are setting the treatment variable to 0
X_treat0 = X_treat.copy()
X_treat0['Smoker'] = 0

# obtain predicted probabilities which equal E(Y|X)
predict1 = logistic.predict_proba(X_treat1)
predict0 = logistic.predict_proba(X_treat0)

# estimate ATE
e_ate = np.mean(predict1 - predict0)
e_ate

3.501176362740954e-21

**Wow, logistic regression gave us an ATE really close to 0.**

Now perform the analysis for type 2 diabetes data.

In [65]:
# split up feature matrix and target array
# X_treat matrix includes the treatment variable smoker 
d2_X_treat =  diabetes2.drop('Diabetes_binary', axis = 1)
d2_y = diabetes2['Diabetes_binary']

# fit logistic regression model
logistic = LogisticRegression(solver='liblinear')
logistic.fit(d2_X_treat, d2_y)

# X_treat1 indicates that we are setting the treatment variable to 1
d2_X_treat1 = d2_X_treat.copy()
d2_X_treat1['Smoker'] = 1

# X_treat0 indicates that we are setting the treatment variable to 0
d2_X_treat0 = d2_X_treat.copy()
d2_X_treat0['Smoker'] = 0

# obtain predicted probabilities which equal E(Y|X)
predict1 = logistic.predict_proba(d2_X_treat1)
predict0 = logistic.predict_proba(d2_X_treat0)

# estimate ATE
e_ate = np.mean(predict1 - predict0)
e_ate

-8.024894418275118e-20

Pretty much same result.